In [3]:
import nltk
from nltk.corpus import cess_esp
nltk.download('conll2002')
nltk.download('cess_esp')
 
tagged_sentences = nltk.corpus.conll2002.tagged_sents()
tagged_sentences1 = cess_esp.tagged_sents()
 
print(tagged_sentences1[0])
print("Tagged sentences: ", len(tagged_sentences))
print("Tagged words:", len(nltk.corpus.conll2002.tagged_words()))

print("Tagged sentences: ", len(tagged_sentences1))
print("Tagged words:", len(cess_esp.tagged_words()))
 

[nltk_data] Downloading package conll2002 to
[nltk_data]     /Users/raulernestogutierrezdepinerezreyes/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!
[nltk_data] Downloading package cess_esp to
[nltk_data]     /Users/raulernestogutierrezdepinerezreyes/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


[('El', 'da0ms0'), ('grupo', 'ncms000'), ('estatal', 'aq0cs0'), ('Electricité_de_France', 'np00000'), ('-Fpa-', 'Fpa'), ('EDF', 'np00000'), ('-Fpt-', 'Fpt'), ('anunció', 'vmis3s0'), ('hoy', 'rg'), (',', 'Fc'), ('jueves', 'W'), (',', 'Fc'), ('la', 'da0fs0'), ('compra', 'ncfs000'), ('del', 'spcms'), ('51_por_ciento', 'Zp'), ('de', 'sps00'), ('la', 'da0fs0'), ('empresa', 'ncfs000'), ('mexicana', 'aq0fs0'), ('Electricidad_Águila_de_Altamira', 'np00000'), ('-Fpa-', 'Fpa'), ('EAA', 'np00000'), ('-Fpt-', 'Fpt'), (',', 'Fc'), ('creada', 'aq0fsp'), ('por', 'sps00'), ('el', 'da0ms0'), ('japonés', 'aq0ms0'), ('Mitsubishi_Corporation', 'np00000'), ('para', 'sps00'), ('poner_en_marcha', 'vmn0000'), ('una', 'di0fs0'), ('central', 'ncfs000'), ('de', 'sps00'), ('gas', 'ncms000'), ('de', 'sps00'), ('495', 'Z'), ('megavatios', 'ncmp000'), ('.', 'Fp')]
Tagged sentences:  35651
Tagged words: 678377
Tagged sentences:  6030
Tagged words: 192685


In [4]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [5]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }
 

In [6]:
from nltk.tag.util import untag
 
# Split the dataset for training and testing
cutoff = int(.75 * len(tagged_sentences1))
training_sentences = tagged_sentences1[:cutoff]
test_sentences = tagged_sentences1[cutoff:]
 
def transform_to_dataset(tagged_sentences1):
    X, y = [], []
 
    for tagged in tagged_sentences1:
        X.append([features(untag(tagged), index) for index in range(len(tagged))])
        y.append([tag for _, tag in tagged])
 
    return X, y
 
X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(test_sentences)
 
print(len(X_train))     
print(len(X_test))         
print(X_train[0])
print(y_train[0])
 
# 2935
# 979
# [{'word': 'Pierre' ...
# ['NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NNP', 'CD', '.']
 

4522
1508
[{'word': 'El', 'is_first': True, 'is_last': False, 'is_capitalized': True, 'is_all_caps': False, 'is_all_lower': False, 'prefix-1': 'E', 'prefix-2': 'El', 'prefix-3': 'El', 'suffix-1': 'l', 'suffix-2': 'El', 'suffix-3': 'El', 'prev_word': '', 'next_word': 'grupo', 'has_hyphen': False, 'is_numeric': False, 'capitals_inside': False}, {'word': 'grupo', 'is_first': False, 'is_last': False, 'is_capitalized': False, 'is_all_caps': False, 'is_all_lower': True, 'prefix-1': 'g', 'prefix-2': 'gr', 'prefix-3': 'gru', 'suffix-1': 'o', 'suffix-2': 'po', 'suffix-3': 'upo', 'prev_word': 'El', 'next_word': 'estatal', 'has_hyphen': False, 'is_numeric': False, 'capitals_inside': False}, {'word': 'estatal', 'is_first': False, 'is_last': False, 'is_capitalized': False, 'is_all_caps': False, 'is_all_lower': True, 'prefix-1': 'e', 'prefix-2': 'es', 'prefix-3': 'est', 'suffix-1': 'l', 'suffix-2': 'al', 'suffix-3': 'tal', 'prev_word': 'grupo', 'next_word': 'Electricité_de_France', 'has_hyphen': Fal

In [7]:
pip install sklearn-crfsuite

Note: you may need to restart the kernel to use updated packages.


In [9]:
## Funcion que permite forzar el uso de GPU cuando estan presentes

import tensorflow as tf
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

ModuleNotFoundError: No module named 'tensorflow'

In [8]:
from sklearn_crfsuite import CRF
model = CRF()
model.fit(X_train, y_train)

/Users/raulernestogutierrezdepinerezreyes/anaconda3/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm=None, all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [11]:
sentence = ['el', 'hombre', 'bajo','canta', 'bajo', 'el', 'puente', 'rojo','en','Sao','Paulo','en','la','escalera', 'baja']
def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, model.predict([sentence_features])[0]))
 
print(pos_tag(sentence))  # [('I', 'PRP'), ('am', 'VBP'), ('Bob', 'NNP'), ('!', '.')]

[('el', 'da0ms0'), ('hombre', 'ncms000'), ('bajo', 'sps00'), ('canta', 'ncfs000'), ('bajo', 'sps00'), ('el', 'da0ms0'), ('puente', 'ncms000'), ('rojo', 'aq0ms0'), ('en', 'sps00'), ('Sao', 'np0000l'), ('Paulo', 'np0000l'), ('en', 'sps00'), ('la', 'da0fs0'), ('escalera', 'ncfs000'), ('baja', 'aq0fs0')]


In [12]:
from sklearn_crfsuite import metrics
 
y_pred = model.predict(X_test)
print(metrics.flat_accuracy_score(y_test, y_pred))
 
# 0.9602683593122289

0.9286018168622838


In [16]:

i=0
while i<= 1:
    print(y_test[i])
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&COMPARAMOS')
    print(y_pred[i])
    i+=1
    

['cs', 'da0fs0', 'ncfs000', 'vaii3s0', 'vsp00sm', 'aq0cs0', 'Fc', 'da0ms0', 'ncms000', 'vmii3s0', 'ncms000', 'sps00', 'vmn0000', 'dp3cs0', 'di0fs0', 'ncfs000', 'Fc', 'pr0cn000', 'rg', 'vmii3s0', 'vmn0000', 'vmg0000', 'da0ms0', 'ncms000', 'sps00', 'da0fs0', 'ncfs000', 'sps00', 'cs', 'sn.e-SUJ', 'p0000000', 'vmsi3s0', 'sps00', 'aq0cs0', 'ncfs000', 'Fp']
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&COMPARAMOS
['cs', 'da0fs0', 'ncfs000', 'vaii3s0', 'vsp00sm', 'aq0cs0', 'Fc', 'da0ms0', 'ncms000', 'vmii3s0', 'ncms000', 'sps00', 'vmn0000', 'dp3cs0', 'di0fs0', 'ncfs000', 'Fc', 'pr0cn000', 'rg', 'vmii3s0', 'vmn0000', 'vmg0000', 'da0ms0', 'ncms000', 'sps00', 'da0fs0', 'ncfs000', 'sps00', 'cs', 'sn.e-SUJ', 'p0300000', 'vmsi3s0', 'sps00', 'aq0cs0', 'ncfs000', 'Fp']
['ncfp000', 'rg', 'aq0fp0', 'cc', 'rg', 'aq0cp0', 'sps00', 'da0fs0', 'ncfs000', 'sps00', 'vmn0000', 'da0ms0', 'ncms000', 'Fp']
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&COMPARAMOS
['np0000p', 'rg', 'ncfp000', 'cc', 'rg', 'aq0cp0', 'sps00', 'da0fs0', 'nc